## haMSM for Eg5-ADP WT Unbinding WE Simulation Data
---
Testing different bin configs and how much iteration info I can use.

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tqdm.auto

In [58]:
pwd

'/ix/lchong/dty7/eg5/h2p-we/multi-mab_nomon_v01/haMSM'

In [59]:
from msm_we import msm_we
import mdtraj as md

In [62]:
import ray
ray.init(num_cpus=8)
#ray.init()

2023-09-01 16:51:29,530	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.11
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8266


In [63]:
#ray.shutdown()

Model Name

In [64]:
model_name = 'Eg5_WT_bin_opt'

In [65]:
import logging
from rich.logging import RichHandler
log = logging.getLogger()
log.addHandler(RichHandler())

msm_log = logging.getLogger("msm_we.msm_we")

In [66]:
def processCoordinates(self, coords):
    log.debug("Processing coordinates")

    if self.dimReduceMethod == "none":
        xt = md.Trajectory(xyz=coords, topology=None)
        indCA = self.reference_structure.topology.select("name CA")
        indAlign = self.reference_structure.topology.select("name CA")
        x2 = xt.superpose(self.reference_structure,atom_indices=indAlign)
        coords = x2._xyz.astype('float64')
        nA = np.shape(indCA)[0]
        nC = np.shape(coords)[0]
        new_coords = coords[:,tuple(indCA),:]
        #new_coords = numpy.double(new_coords)
        #new_coords = coords[:, :923, :]
        data = new_coords.reshape(nC, 3 * nA)
        model.nAtoms = nA
        return data

    if self.dimReduceMethod == "pca" or self.dimReduceMethod == "vamp":

        # Dimensionality reduction
        
        xt = md.Trajectory(xyz=coords, topology=None)
        # ADP heavy atoms
        indADP = self.reference_structure.topology.select("not type H and resid 370")
        # CA atoms of loop 5
        indCA = self.reference_structure.topology.select("name CA and resid 116 to 133")
        # CA atoms of Eg5 (non-flexible termini)
        #indCA = self.reference_structure.topology.select("name CA and resid 18 to 364")
        # calc distance matrix
        pair1, pair2 = np.meshgrid(indADP, indCA, indexing="xy")
        
        ### for 2D distance matrices ###
        #pair1, pair2 = np.meshgrid(indCA, indCA, indexing="xy")
        #indUT = np.where(np.triu(pair1, k=1) > 0)
        #pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
        #dist = md.compute_distances(xt, pairs, periodic=True, opt=True)
        
        ### special for 1D distance matrix of ADP-Eg5 ###
        # make pair array
        pairs = np.array((pair1, pair2))
        # reduce dim to 2
        pairs = pairs.squeeze()
        # convert to shape (n_pairs, 2) where row items are the atom index pair vals
        pairs = np.rot90(pairs, k=-1)
        #indUT = np.where(np.triu(pair1, k=1) > 0)
        #pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
        dist = md.compute_distances(xt, pairs, periodic=True, opt=True)

        return dist

# Monkey-patch, i.e. replace the placeholder processCoordinates method in msm_we.modelWE
msm_we.modelWE.processCoordinates = processCoordinates

Testing different `processCoordinates` code. Needs to be adjusted to handle non-2D distance matrix type data.

In [9]:
#xt = md.load("eg5_2022_dry.pdb", top="eg5_2022_dry.pdb")
#xt = md.Trajectory(xyz=coords, topology=None)
# ADP heavy atoms
#indADP = xt.topology.select("not type H and resid 370")
# CA atoms of Eg5 (non-flexible termini)
#indCA = xt.topology.select("name CA and resid 18 to 364")

#pair1, pair2 = np.meshgrid(indADP, indCA, indexing="xy")
#pair1, pair2 = np.meshgrid(indCA, indCA, indexing="xy")

# make pair array
#pairs = np.array((pair1, pair2))
# reduce dim to 2
#pairs = pairs.squeeze()
# convert to shape (n_pairs, 2) where row items are the atom index pair vals
#pairs = np.rot90(pairs, k=-1)
#indUT = np.where(np.triu(pair1, k=1) > 0)
#pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
#dist = md.compute_distances(xt, pairs, periodic=True, opt=True)

Applying sigmoidal transform:

Trying it with 10A cutoff since using ADP to L5 distances.

In [10]:
def sigmoidal(x, cutoff=10):
    """
    $f(x) = 1 - \frac{1}{1 + \exp\frac{-(x-cutoff)}{0.75}}$
    
    Parameters
    ----------
    x : input float
    
    Returns
    -------
    f(x) : sigmoidal function transformed float
    """
    return 1 - (1 / (1 + np.exp(-(x - cutoff) / 0.75)))

In [11]:
# vectorize sigmoid function to apply to entire dmat
# this should work to apply to multi-dimensional arrays
vf = np.vectorize(sigmoidal)

In [12]:
def processCoordinates(self, coords):
    log.debug("Processing coordinates")

    if self.dimReduceMethod == "none":
        xt = md.Trajectory(xyz=coords, topology=None)
        indCA = self.reference_structure.topology.select("name CA")
        indAlign = self.reference_structure.topology.select("name CA")
        x2 = xt.superpose(self.reference_structure,atom_indices=indAlign)
        coords = x2._xyz.astype('float64')
        nA = np.shape(indCA)[0]
        nC = np.shape(coords)[0]
        new_coords = coords[:,tuple(indCA),:]
        #new_coords = numpy.double(new_coords)
        #new_coords = coords[:, :923, :]
        data = new_coords.reshape(nC, 3 * nA)
        model.nAtoms = nA
        return data

    if self.dimReduceMethod == "pca" or self.dimReduceMethod == "vamp":

        # Dimensionality reduction
        
        xt = md.Trajectory(xyz=coords, topology=None)
        # ADP heavy atoms
        indADP = self.reference_structure.topology.select("not type H and resid 370")
        # CA atoms of loop 5
        indCA = self.reference_structure.topology.select("name CA and resid 116 to 133")
        # CA atoms of Eg5 (non-flexible termini)
        #indCA = self.reference_structure.topology.select("name CA and resid 18 to 364")
        # calc distance matrix
        pair1, pair2 = np.meshgrid(indADP, indCA, indexing="xy")
        
        ### for 2D distance matrices ###
        #pair1, pair2 = np.meshgrid(indCA, indCA, indexing="xy")
        #indUT = np.where(np.triu(pair1, k=1) > 0)
        #pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
        #dist = md.compute_distances(xt, pairs, periodic=True, opt=True)
        
        ### special for 1D distance matrix of ADP-Eg5 ###
        # make pair array
        pairs = np.array((pair1, pair2))
        # reduce dim to 2
        pairs = pairs.squeeze()
        # convert to shape (n_pairs, 2) where row items are the atom index pair vals
        pairs = np.rot90(pairs, k=-1)
        #indUT = np.where(np.triu(pair1, k=1) > 0)
        #pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
        dist = md.compute_distances(xt, pairs, periodic=True, opt=True)

        return vf(dist)

# Monkey-patch, i.e. replace the placeholder processCoordinates method in msm_we.modelWE
msm_we.modelWE.processCoordinates = processCoordinates

### Building block-wise haMSM
---

In [13]:
cat ../tstate.file

unbound 12 1 6.1 6.1


Create the model object and initialize it

In [14]:
model =  msm_we.modelWE()
model.initialize(
    fileSpecifier=['west-packopt.h5'], 
    refPDBfile = 'eg5_2022_dry.pdb',
    modelName = model_name,
    basis_pcoord_bounds  = [[0,10], [100,np.inf], [0,6], [0,6]],
    target_pcoord_bounds = [[12,np.inf], [-np.inf,100], [6,np.inf], [6,np.inf]],
    dim_reduce_method = 'pca',
    tau = 50e-12,
    pcoord_ndim = 4,
)

Load all coords and pcoords up to the last iteration you want to use for analysis with:

TODO: try using larger coord set (entire 1-585?)

In [15]:
first_iter = 400
last_iter = 585

In [16]:
model.get_iterations()
# use up to n last_iter iterations
model.get_coordSet(last_iter=last_iter, streaming=True)

Output()

Prepare dimensionality reduction transformer by running

In [17]:
model.dimReduce()

Output()

In [67]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_postPCA.pickle', 'rb') as f:
    model = load(f)

In [18]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_postPCA.pickle', 'wb') as f:
    dump(model, f)

Get WE bin mapper for cluster stratification (using updated bins, from looking at clustering results and from looking at evolution plots).

In [68]:
import westpa
from westpa.core.binning import RectilinearBinMapper

# 1) ADP-Mg RMSD
# 2) ADP-Mg and Eg5 IntEne
# 3) ADP and Eg5 min dist
# 4) ADP-PO4 and Eg5 min dist
new_mapper = RectilinearBinMapper([[0, 2, 6, 11, 13, 15, 
                                    17, 20, 30, np.inf],
                                   [-np.inf, -200, -100, -50, 0,
                                    50, 100, 200, 270, np.inf],
                                   [0, 3, 6, np.inf],
                                   [0, 3, 6, np.inf],
                                  ])

Cluster and store original clustering

In [69]:
model.cluster_coordinates(n_clusters = 10, 
                          stratified = True,
                          tol = 1e-5, 
                          streaming = True, 
                          use_ray = True, 
                          first_cluster_iter = first_iter,
                          user_bin_mapper = new_mapper, 
                          store_validation_model = True)

[09/01/23 16:51:37] INFO     Be aware: Number of cluster centers is an important parameter, and  ]8;id=615595;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=508740;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py#160\160]8;;\
                             can drastically affect model quality. We recommend examining                          
                             block-validation results with a range of numbers of clusters, to                      
                             check for overfitting.                                                                

                    INFO     Loading user-specified bin mapper for stratified clustering.        ]8;id=954139;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=704853;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py#585\585]8;;\

                    INFO     Doing stratified clustering with 7290 total clusters       ]8;id=234396;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/stratified_clustering.py\stratified_clustering.py]8;;\:]8;id=563514;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/stratified_clustering.py#66\66]8;;\

Output()

In [14]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap.pickle', 'rb') as f:
    model = load(f)

In [70]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap.pickle', 'wb') as f:
    dump(model, f)

In [71]:
model.n_clusters

7290

Create the flux matrix with

In [72]:
model.get_fluxMatrix(n_lag=0, first_iter=first_iter, last_iter=last_iter, use_ray=True)

Output()

In [73]:
model.fluxMatrix

In [8]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postfluxget.pickle', 'rb') as f:
    model = load(f)

In [74]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postfluxget.pickle', 'wb') as f:
    dump(model, f)

Clean disconnected states and sort the flux matrix with

In [75]:
# optionally could adjust this cluster class attribute for pcoord sorting selection (default=0)
#model.pcoord_to_use = 0

In [76]:
model.organize_fluxMatrix(use_ray=True)

[09/01/23 17:45:39] INFO     Started with 7290 clusters, and removed 6590                       ]8;id=87533;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=486882;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py#1050\1050]8;;\

                    WARNING  All clusters were cleaned from bins {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, ]8;id=650544;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=762829;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py#1067\1067]8;;\
                             10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,                       
                             26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,                       
                             42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,                       
                             58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,                       
                             74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,                       
                             90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104,                      
                             105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,                      
                             119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132,                      
                             133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,                      
                             146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,                      
                             159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 172,                      
                             173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186,                      
                             187, 188, 191, 192, 195, 196, 197, 201, 204, 205, 206, 209, 210,                      
                             213, 214, 215, 218, 219, 221, 222, 223, 224, 226, 227, 228, 229,                      
                             230, 231, 232, 233, 235, 236, 237, 238, 239, 240, 241, 242, 243,                      
                             244, 245, 246, 247, 248, 249, 250, 251, 254, 255, 257, 258, 259,                      
                             260, 263, 264, 266, 267, 268, 269, 272, 273, 276, 277, 278, 281,                      
                             282, 285, 286, 287, 290, 291, 293, 294, 295, 296, 299, 300, 301,                      
                             302, 303, 304, 305, 307, 308, 309, 310, 311, 312, 313, 314, 316,                      
                             317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329,                      
                             330, 331, 332, 335, 336, 338, 339, 340, 341, 344, 345, 347, 348,                      
                             349, 350, 353, 354, 357, 358, 359, 362, 363, 366, 367, 368, 370,                      
                             371, 372, 373, 374, 375, 376, 377, 379, 380, 381, 382, 383, 384,                      
                             385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397,                      
                             398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410,                      
                             411, 412, 413, 416, 417, 419, 420, 421, 422, 425, 426, 429, 430,                      
                             431, 434, 435, 438, 439, 440, 444, 447, 448, 449, 450, 451, 452,                      
                             453, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466,                      
                             467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479,                      
                             480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492,                      
                             493, 494, 497, 498, 500, 501, 502, 503, 504, 506, 507, 510, 511,                      
                             512, 513, 515, 516, 519, 520, 521, 522, 524, 525, 528, 529, 530,            

Output()

Output()

[09/01/23 18:20:23] INFO     Note: Sorting bins, assuming that pcoord 0 is meaningful for       ]8;id=11450;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=406367;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_clustering.py#1643\1643]8;;\
                             sorting                                                                               

In [77]:
model.fluxMatrix

array([[9.66747608e-086, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 1.45012141e-085, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 4.38057510e-086, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        4.32920247e-108, 0.00000000e+000, 1.80332818e-110],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 7.63145788e-057]])

In [7]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postfluxorg.pickle', 'rb') as f:
    model = load(f)

In [78]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postfluxorg.pickle', 'wb') as f:
    dump(model, f)

### Begin Model Analysis
---

Normalize the flux matrix to produce a transition matrix with

In [79]:
model.get_Tmatrix()

In [80]:
model.Tmatrix

array([[0.33333333, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00352198, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.41099696, 0.        ,
        0.00171201],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

Obtain steady-state distribution with

In [81]:
model.get_steady_state()

[09/01/23 18:20:35] INFO     Negative elements in pSS after normalization, attempting to correct   ]8;id=516223;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=480304;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py#238\238]8;;\
                             with matrix power method.                                                             

                    WARNING  Power method did NOT obtain semidefinite pSS. Some negative values    ]8;id=737550;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=292367;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py#256\256]8;;\
                             remain. This is weird, and you should try to figure out why this is                   
                             happening.                                                                            

                    WARNING  Some negative  elements in pSS... Ignoring, and setting model.pSS     ]8;id=174074;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=57348;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py#276\276]8;;\
                             anyways.                                                                              

[09/01/23 18:20:36] INFO                                                                           ]8;id=677231;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=34953;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py#179\179]8;;\
                             Flux converged to 9.9720e-29 after 3 iterations of inverse iteration.                 

(array([101, 165, 221, 249, 259, 274, 283, 379]),)


AssertionError: Negative elements in pSS

In [ ]:
model.update_cluster_structures()

In [ ]:
model.get_steady_state_target_flux()

print(f'Steady-state target rate is {model.JtargetSS:.2e}')

In [8]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postsstflux.pickle', 'rb') as f:
    model = load(f)

In [ ]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postsstflux.pickle', 'wb') as f:
    dump(model, f)

### Begin Model Validation
---

In [ ]:
# shape: clusters, pcoords, min/max
model.targetRMSD_minmax.shape

In [ ]:
# For checking just in case your bins are not good.
array = model.targetRMSD_minmax[:,0,:]
print(array[23],array[50],array[132],array[142],array[148],array[151])

In [ ]:
# one issue here: each pcoord dim is the same values as pcoord 0 
# (TODO: update code for grabbing all pcoords)
model.targetRMSD_minmax[0]

In [ ]:
model.targetRMSD_minmax[:,0,:].shape

In [ ]:
def plot_clusters(pcoord_to_use=0, ax=None):
    """
    MSM Bin coverage plot with sorting based on specified pcoord.
    """
    # sort by pcoord n
    model.pcoord_to_use = pcoord_to_use
    model.get_cluster_centers()
    #model.update_sorted_cluster_centers()

    # isolate pcoord min max array
    array = model.targetRMSD_minmax[:,pcoord_to_use,:]

    # set up plot objects
    if ax is None:
        fig, ax = plt.subplots(figsize=(10,5))
    else:
        fig = plt.gcf()
    
    # len-2 because the last two are NaN (eq simulation; target/basis states)
    for idx in range(0,len(array)):
        ax.errorbar(array[idx,0], idx,xerr=[[0],[array[idx,1]-array[idx,0]]])
        #ax.plot(idx, array[idx,1], 'ro')
    ax.set_ylabel('cluster number')
    ax.set_xlabel(f'Pcoord {pcoord_to_use}')
    ax.set_title('MSM Bins Coverage')

In [ ]:
plot_clusters(0)

In [ ]:
plot_clusters(1)

In [ ]:
# making a 4 panel plot for each pcoord dim
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(10,5))
for i, panel in enumerate(ax.reshape(-1)):
    plot_clusters(i, panel)
    model.draw_basis_target_boundaries(panel, i)
ax[0,1].set_ylabel("")
ax[1,1].set_ylabel("")
ax[1,0].set_title("")
ax[1,1].set_title("")
plt.subplots_adjust(hspace=0.35)

Had to adjust `_plotting.py` before `plot_flux` worked:
``` Python
322             binCenters[_model.indTargets] = _model.target_bin_centers[pcoord_to_use]
323             binCenters[_model.indBasis] = _model.basis_bin_centers[pcoord_to_use]
```

In [ ]:
model.plot_flux(suppress_validation=True, pcoord_to_use=1)

In [ ]:
# making a 4 panel flux vs pcoord plot for each pcoord dim
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(10,5))
for i, panel in enumerate(ax.reshape(-1)):
    model.plot_flux(suppress_validation=True, pcoord_to_use=i, ax=panel)
    
plt.subplots_adjust(hspace=0.35)

In [ ]:
model.get_committor()

In [ ]:
# optionally could adjust this cluster class attribute for pcoord to use (default=0)
model.pcoord_to_use = 0

In [ ]:
model.plot_committor()

In [ ]:
# making a 4 panel committor plot for each pcoord dim
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(10,5))
for i, panel in enumerate(ax.reshape(-1)):
    panel.scatter(model.targetRMSD_centers[:,i], model.q, s=15, c='k')
    panel.set_xlabel(f"pcoord {i}")
    model.draw_basis_target_boundaries(panel, i)
ax[0,0].set_ylabel("pseudocommittor to target")
ax[1,0].set_ylabel("pseudocommittor to target")
fig.subplots_adjust(hspace=0.35)
#plt.yscale('log')

In [ ]:
# first 3 center datapoints using all pcoord dims
model.targetRMSD_centers[:3,:]

In [ ]:
# q is committor array per cluster
model.q.shape

In [ ]:
model.plot_flux_committor(suppress_validation=True)
#plt.gca().set_yscale('linear')
#plt.savefig('flux_pseudo_uneven_bin-restart34-t2-c18.png')

In [ ]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postvalidation.pickle', 'rb') as f:
    model = load(f)

In [ ]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postvalidation.pickle', 'wb') as f:
    dump(model, f)

### Post-featurization Steps: Block Validation
---

In [ ]:
model.do_block_validation(cross_validation_groups=2, cross_validation_blocks=8, use_ray=True)

In [ ]:
# plot again with cross validated model
model.plot_flux_committor()

In [8]:
# For loading the clustered model data
from pickle import load
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postblockvalidation.pickle', 'rb') as f:
    model = load(f)

In [ ]:
# For saving the clustered model data
from pickle import dump
with open(f'{model_name}_{first_iter}-{last_iter}_4Dmap_postblockvalidation.pickle', 'wb') as f:
    dump(model, f)

#### Block validation rates

In [ ]:
for i in range(len(model.validation_models)):
    model.validation_models[i].get_steady_state_target_flux()

    print(f'Steady-state target rate is {model.validation_models[i].JtargetSS:.2e} for model {i}')

In [ ]:
list_of_rates = [model.JtargetSS, model.validation_models[0].JtargetSS, model.validation_models[1].JtargetSS]
print(np.std(list_of_rates))
print(np.average(list_of_rates))

#### Quick aside: looking at the theory and code behind $k_{AB}$ calculation

Normalizing `model.fluxMatrix` to get transition matrix, output `model.Tmatrix`.
* row-normalize the flux matrix
* set any states with 0 flux (i.e. sinks) to have 1.0
* set target bins to uniformly recycle into basis bins

In [11]:
model.get_Tmatrix()

First calculate SS probability of microbins based on SS solution of the transition matrix (`model.pSS`). 

These are calculated as the eigenvectors of the transition matrix.

In [12]:
model.get_steady_state_algebraic()

[06/15/23 12:11:28] INFO     Negative elements in pSS after normalization, attempting to correct   ]8;id=623078;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=966892;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py#237\237]8;;\
                             with matrix power method.                                                             

                    INFO     Corrected to semidefinite pSS in 176 iterations                       ]8;id=395661;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=933098;file:///ihome/lchong/dty7/Apps/msm_we/msm_we/_hamsm/_analysis.py#248\248]8;;\

Calculate sum of target state ss flux ($k_{AB}$) via https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8045600/#FD3 equation 3:

$k_{AB} = Flux(A \rightarrow B;SS) = \sum_{i\notin B, j\in B} p_{i}^{\alpha}T_{ij}^{\alpha}$ (haMSM)

Here, $p^{\alpha}$ is given by `model.pSS` and $T^{\alpha}$ is given by `model.Tmatrix`.

Running `model.get_steady_state_target_flux()` calculates $k_{AB}$, output `model.JtargetSS`.

In [13]:
model.get_steady_state_target_flux()

In [14]:
model.JtargetSS

1.9954185219363416e-21

### Getting model outputs
---

Block validation output

In [ ]:
import weight_loop
for idx, v_model in enumerate(model.validation_models):
    if idx == 0:
        continue
    weight_loop.create_reweighted_h5_global(v_model,new_name=f'west_{idx}.h5')

Use haMSM weights to create new west.h5 file, use this updated west.h5 file for running subsequent WE simulations.

In [ ]:
import weight_loop

weight_loop.create_reweighted_h5_global(model, west_name='west_nocoords.h5', copy=True, 
                                        struct_filetype='ncrst', new_name=f'west_reweight.h5', 
                                        gen_sstates=True, pdb_out=False, link_out=True, 
                                        link_path="/ocean/projects/mcb180038p/jml230/bdpa_wsh2029_p3_r3",)

Extract transition state (pseudo-committor of a certain range)